# **Blob analysis**
Once foreground/background segmentation has been accomplished, usually next task is the analysis of the individual foreground objects to get some high-level knowledge on the observed scene.

Such knowledge may pertain detecting what types of objects are contained in the scene, for example: measuring their position and orientation, assesing if they show or not manifacturing faults, etc...

The individial objects, referred as **blobs** (Binary Large Objects), need to be isolated within the overall foreground region.
This step is known as **connected components labelling**.

Indvidual objects can be then processed to extract specific *features* (the properties of the blob) related to the required kind of high-level knowledge.
These featurese may be computed from all the pixels belonging to the blob (*region features*) or from boundary pixels only (*contour features*).
This process is known as **blob analysis**.

*Shape-related* features need to exihibit **invariance to the transformations** that the image of the object may undergo during the analysis.
Moreover, mostly shape-related features need to fulfil **invariance to similarity transformation** (translation, rotation and scale change).
In particular, they **do not** have to change if the object appears at different position, rotated or shows a different size in the image.

Object detection based on blob features can be accomplished either by handcrafted rules or based on machine learning techniques (classification to map a feature vector into a set of classes).

*Example*: quality process and OCR (a very widespread paradigm):
1. Grab image;
1. ROI (relative ROI, anchor to establish a relative position that has a fix pattern);
1. Binarization;
1. Labelling (usuallly with different gray-scales or pseudo-colors);
1. Blob analysis.

## Distance and connectivity
To discuss labelling algorithm, the notion of **connectivity**, related on the **distance** on the discrete plane $E^2$, is required.

Given $p_1(i_1,j_1),p_2(i_2,j_2),p_3(i_3,j_3) \in E^2$, $D$ is a distance if:
1. It's non negative: $D(p_1,p_2) \geq 0, \quad D(p_1,p_2) = 0 \iff p_1=p_2$;
1. It's symmetric: $D(p_1,p_2) = D(p_2,p_1)$;
1. It holds the triangular inequality (better go straight): $D(p_1,p_3) \leq D(p_1,p_2) + D(p_2,p_3)$.

The two main distances defined on the discrete plane $E^2$ are:
1. **City-block distance** (or Manhattan, or taxi-cab):
$$D_4 = |i_1 - i_2|+|j_1 - j_2|$$
According to the definition, a pixel can be reached from another only with horizontal or vertical shifts.
The set of point having $D_4 \leq r$ from a given one is a rhombus with diagonal of lenght $2r + 1$:
<center> <img src=https://i.ibb.co/H4TMz84/photo-2021-01-23-10-24-14.jpg width="500px" /> </center>
Defined as neighbours of $p$ the set of points having $D=1$ from $p$, using the $D_4$ we'll have the so called 4-neighbourhood of $p$, denoted as $n_4(p)$.

1. **Chessboard distance**:
$$D_8(p_1,p_2) = \max(|i_1 - j_1|,|j_1 - j_2|)$$
According to the definition, horizontal, vertical and even diagonal shifts have the same weight.
The set of point having $D_8 \leq r$ from a given one is a square with side of lenght $2r + 1$:
<center> <img src=https://i.ibb.co/cX7Dp9M/photo-2021-01-23-10-24-14.jpg width="500px" /> </center>
Defined as neighbours of $p$ the set of points having $D=1$ from $p$, using the $D_8$ we'll have the so called 8-neighbourhood of $p$, denoted as $n_8(p)$.

## Connected components of a binary image
A path of lenght $n$ from pixel $p$ to pixel $q$ is a *sequence of pixels* (so there's an order) $p = p_1, p_2, \dots , p_n = q$, such as $p_i$ and $p_{i+1}$ (consecutive pixels) are neighbours according to the chosen distance (both discrete distances have the same lenght as the smallest path from $p$ to $q$).

A set of pixels $R$ is said to be a **connected region**, if for any two pixels $p,q$ in $R$, exists a path contained in $R$ between $p$ and $q$.
Depending on the chosen distance, the region is said to be $4$-connected or $8$-connected.

A set of pixels is said to be a *connected foreground* (or background) *region* if it's a connected region and contains foreground (or background) pixels only.

A **connected component** of a binary image is a **maximal connected foreground region**.

Computation with pixels belonging to different connected components are given different labels, while background pixels are unaffected:
<center> <img src=https://i.ibb.co/9n2F9r8/photo-2021-01-23-10-50-51.jpg width="700px" /> </center>

## The classical 2-scan algorithm
There's exist a tons of labelling algorithm, with different speed and efficiency. This in particular is a sequential algorithm.

* By the **first scan** foreground pixels take temporary labels, based on those given by already visited neighbours, which depends on the chosen distance and the scan order.

After the first scan, different blobs have certainly been given different labels. 
Depending on the shape, this could happens for connected parts of the same blob too. 
This is why a second scan is required.

* The **second scan** allows an unique final label to be assigned to those parts (that belong to the same blob), that had been given different temporary labels by the first scan.

Equivalent temporary labels need to be found between the two scans, so as to assign a unique final label to each of the equivalence classes among temporary labels.

* **First scan**
<center> <img src=https://i.ibb.co/RN4mk2Z/photo-2021-01-23-11-06-48.jpg width="700px" /> </center>

1. If two neighbours pixels $p,q$ are both labelled as background (B), the the pixel $x$ it's the first pixel of a new object found, and so we assign to him a new label;
2. If $p$ belongs to the backgrounf and $q$ has been labelled with $l_q$, then also $x$ will be labelled as $l_q$;
3. Viceversa;
4. If two neighbours pixels $p,q$ are both labelled as different from background, and differents among them too, then we can assign to $x$ either $l_q$ or $l_p$, but moreover we have to add the **class equivalence** of these two labels $\{l_q,l_p\}$. 
This is the case when a pixel connects two parts of the same blob (considered at the moment distinct objects).

So, after the first scan, some labels are into class equivalences, and with the second scan we choose the right label.

*Example*
<center> <img src=https://i.ibb.co/qFh5rqv/photo-2021-01-23-11-22-24.jpg width="800px" /> </center>


### Handling equivalences
How do we store pairwise equivalences? During the first scan, the equivalences found betweem temporary label pairs are recorded into an $N \times N$ matrix, with $N$ being the number of temporary labels.
<center> <img src=https://i.ibb.co/7t4k1XX/photo-2021-01-23-11-28-20.jpg width="800px" /> </center>

Since it's an equivalence relation the matrix $B$ is symmetric and has unitary main diagonal.

Ater the first scan, $B$ is processed to elicit the equivalence among temporary labels, implied due the transitive property:
$$ \{ i,j \} \to \{ i,k \} \quad \forall \{ k,j \} $$
then:
$$ B[i,k] = B[i,k] \text{ OR } B[k,i] \quad \forall k = 1, \dots , N$$

Thus, through a scan by columns, all the equivalence implied by the transitive property can be recorded into $B$.

Having found equivalence classes, it's necessary to assign an unique final label to each of them. 
For doing so, a simple table (vector) initialized by the identity function may be deployed. 
Then, the second scan boils down to an image look-up operation by the table.

*Long-ass example*
<center> <img src=https://i.ibb.co/W3pqsHt/photo-2021-01-23-11-41-53.jpg width="800px" /> </center>

## Area and barycenter
Once regions have been identified, features associated with each region are computed.
The simplest ones are:
* **Area**: amount of pixels belonging to that region $A = \sum_{p \in R} 1$;
* **Barycenter**: center of mass of the region, seen as a set of particle of unitary mass. Let a generic pixel be:
$$p = \begin{bmatrix} i \\ j \end{bmatrix}$$
The barycenter will be:
$$B = {\begin{bmatrix} i_b \\ j_b \end{bmatrix}} = {\begin{bmatrix} \frac{1}{A} \sum_{p \in R} i \\ \frac{1}{A} \sum_{p \in R} j \end{bmatrix}}$$

*Note*: $R$ is a blob. The axis are:

<img src=https://i.ibb.co/1Xv7n2v/photo-2021-01-23-14-21-01.jpg width="250px" />

## Perimeter
The perimeter is defined as the lenght of the contour of the region (blob), that depends on the pixels of the region that belong to the contour.

A pixel $p$, that belong to a region, belong to the contour of that region if **exists at least one background pixel $q$ between its neighbours** (so it depends on the connectivity).

It can be easily observed that:
* $C_4$ is a 8-connected curve $\exists q \in n_4(p) \to è \in C_4$;
* $C_8$ is a 4-connected curve $\exists q \in n_8(p) \to è \in C_8$;

So, the perimeter can be estimated as:
$$P_8 = \sum_{p \in C_4} 1$$
$$P_4 = \sum_{p \in C_8} 1$$
$P_8$ tends to understimate the true lenght, while $P_8$ tends to overstimate it:
<center> <img src=https://i.ibb.co/DR9pZwN/photo-2021-01-23-14-30-26.jpg width="800px" /> </center>

It's then possible to average the two measurements: $\tilde{P} = \frac{P_4 + P_8}{2}$.

*Note*: if the blob has holes we need to find both inner and outer perimeters.

Considering the $C_4$ (8-connected contour), a more accurate approximation can be obtained taking into account the true lenght of diagonal pixels:
$$\tilde{P}_8 = \sum_{p_k \colon p_{k + 1} \in n_4(p_k)} 1 + \sum_{p_k \colon p_{k + 1} \in (n_8(p_k) - n_4(p_k))} \sqrt{2} $$

## Compactness (or form factor)
Represents how much perimeter is needed to contain the area:
$$C = \frac{P^2}{A}$$
It's a dimensionless quantity and so it's scale-invariant.

## Haralick's circularity
In discrete domain, compactness takes its smallest valued not for a circle but for an octagon (8-connectivity) o for a diamond (4-connectivity).
For that reason Haralick proposed this variant feature.

Let's consider:
* The set of point of the contour: $C = \{p_1 , p_2, \dots, p_m\}$;
* The generic pixel: $p_k = \begin{bmatrix} i_k \\ j_k \end{bmatrix}$;
* The baricenter: $B = \begin{bmatrix} i_b \\ j_b \end{bmatrix}$;
* The distance from a contour point to the barycenter: $d_k = \sqrt{(i_k - i_b)^2 +(j_k - j_b)^2}$;
* The mean distance: $\mu_r = \frac{1}{m} \sum_{k = 1}^m d_k$;
* The standard deviation of distances: $\sigma_r^2 = \frac{1}{m} \sum_{k = 1}^m (d_k - \mu_r)^2$

Then, the circularity will be:
$$\tilde{C} = \frac{\mu_R}{\sigma_R}$$
It's once again a scale-invariant features since it's dimensionless.
The smaller the standard deviation (distance are more similar, like in a circle), the higher is the circularity.

## Euler number
It useful to count the number of the holes of a blob.
It's defined as:
$$E = C - H$$
where $C$ is the number of connected component and $H$ is the number of holes.

In practice $E$ is computed for each connected components, to determine the number of its holes ($E=0$ means that the blob has one hole).
$E$ is a topological feature, and so it invariant to the *rubber-sheet transformation*.

*Example*
<center> <img src=https://i.ibb.co/Gn2TbMZ/photo-2021-01-23-14-52-53.jpg width="400px" /> </center>




## Moments
The moment of order $(m,n)$ of a region is defined as:
$$M_{m,n} = \sum_{p \in R} i^m j^n$$
<center> <img src=https://i.ibb.co/kxCDPrs/photo-2021-01-24-09-00-30.jpg width="300px" /> </center>

*Note*: the area is the moment of order $(0,0)$: $M_{0,0} = \sum_{p \in R} i^0 j^0 = \sum_{p \in R} 1 = A$.

The orther relevant moments are the second moments ($m + n = 2$):
* Moment of inertia wrt the $i$-axis: $M_{0,2} = \sum_{p \in R} i^0 j^2 = \sum_{p \in R} j^2$;
* Moment of inertia wrt the $j$-axis: $M_{2,0} = \sum_{p \in R} i^2 j^0 = \sum_{p \in R} i^2$;
* Deviation moment of inertia: $M_{1,1} = \sum_{p \in R} i^1 j^1 = \sum_{p \in R} i j$

Moments change according to the position of the region in the image.
Invariance to translation can be achieved by calculating the moments relative to the barycentre, and they're the so called **central moments**:
$$M^{'}_{m,n} = \sum_{p \in R} (i - i_b)^m (j - j_b)^n$$

To achieve scale-invariance too, normalization is needed:
$$V_{m,n} = \frac{M^{'}_{m,n}}{A^\alpha} \quad \quad \quad \alpha = \frac{m + n}{2} + 1$$

*Example*
<center> <img src=https://i.ibb.co/Wzb4G8h/photo-2021-01-24-09-16-18.jpg width="700px" /> </center>

### Hu's moments
Hu has shown that shape features invariant to rotation, translation and scaling can be define based on normalized central moments. 
He proved the invariance in continuous spacea, but even if computed on blobs the quantities turn out reasonably stable too.

## Orientation
Many applications, such as roboto guidance, require to determine the orientation of the objects appearing in the image.
If an object is somewhat elongated, its orientation can be defined according to the **direction of the axis of least inertia**, referred as **major axis**:
$$\text{major axis} = \text{arg} \min_l \sum_{p \in R} d_l^2 (p)$$

The orientation is typically determined as the angle $\theta$ between the major axis and the horizontal axis $j$ (it can be determined modulo $\pi$).
<center> <img src=https://i.ibb.co/gT2NLyk/photo-2021-01-24-09-28-25.jpg width="300px" /> </center>


### Distance from a point to a line
Let's consider:
* A line parametrized in the $j,i$ plane $l \colon aj +bi + c = 0$;
* A generic point $p = \begin{bmatrix} i \\ j \end{bmatrix}$.

The squared distance from a point to a line can be expressed as:
$$d_l^2(p) = \frac{(ai + bj +c)^2}{a^2 + b^2}$$

<center> <img src=https://i.ibb.co/5R6FmmL/photo-2021-01-24-09-35-54.jpg width="400px" /> </center>

Let be:
$$\tilde{p} = \begin{bmatrix} \tilde{i} \\ \tilde{j} \end{bmatrix} \quad l = \begin{bmatrix} a \\ -b \end{bmatrix} \quad n = \begin{bmatrix} b \\ a \end{bmatrix}$$
Then we can calcukate the distance as:
$$n \cdot (p - \tilde{p}) = |n| |p - \tilde{p}| \cos \omega$$
Substituting we will have:
$$|p - \tilde{p}| = d_l(p) = \frac{n \cdot (p - \tilde{p})}{|n| \cos \omega} = \frac{b(i - \tilde{i}) + a(j - \tilde{j})}{\sqrt{a^2 + b^2} \cos \omega} = \frac{aj + bi + c}{\sqrt{a^2 + b^2} \cos \omega}$$

Since $n$ and $(p - \tilde{p})$ are parallel vectors, then $\cos \omega = \pm 1$.
This means that $d_l(p)$ represents a **signed distance**, depending on $p$ lying on either of the two sides wrt the line.

The squared distance will be:
$$d_l^2(p) = \frac{(aj + bi + c)^2}{a^2 + b^2}$$

### Line through the barycenter and search for axes
Let's consider:
$$p = \begin{bmatrix} i \\ j \end{bmatrix} \quad B = \begin{bmatrix} i_b \\ j_b \end{bmatrix} $$

$$ \bar{p} = \begin{bmatrix} \bar{i} \\ \bar{j} \end{bmatrix} = \begin{bmatrix} i - i_b \\ j - j_b \end{bmatrix}$$

$$\bar{l} = \begin{bmatrix} - \sin \theta \\ \cos \theta \end{bmatrix} = \begin{bmatrix} \alpha \\ \beta \end{bmatrix}$$

<center> <img src=https://i.ibb.co/zxKnz9g/photo-2021-01-24-10-01-10.jpg width="400px" /> </center>

Let's consider the generic parametrized line $\ell$:
$$\bar{\ell} \colon \bar{p} = \lambda \bar{l} \quad \to \quad \begin{cases} \bar{i} = \lambda \alpha \\ \bar{j} = \lambda \beta \end{cases} \quad \to \quad \frac{\bar{j}}{\bar{i}} = \frac{\beta}{\alpha} \quad \to \quad \alpha \bar{j} - \beta \bar{i} = 0$$

Then, the distance between point and line will be:
$$d_l^2(p) = \frac{(aj + bi + c)^2}{a^2 + b^2} \colon \quad \begin{cases} a = \alpha \\ b = - \beta \\ c = 0 \\ a^2 + b^2 = \alpha^2 + \beta^2 = \sin^2 \theta + \cos^2 \theta = 1 \end{cases} \quad \to \quad d_l^2(p) = (\alpha \bar{j} - \beta \bar{i})^2 $$

The moment of inertia wrt the above line, thorugh the barycenter, can be expressed as:
$$M(\bar{l}) = \sum_{p \in R} d_l^2(p) = \sum_{p \in R} (\alpha \bar{j} - \beta \bar{i})^2 = \alpha^2 \sum_{p \in R}\bar{j}^2 - 2 \alpha \beta \sum_{p \in R}\bar{i} \bar{j} + \beta^2 \sum_{p \in R}\bar{i}^2$$

Since:
* $M^{'}_{0,2} = \sum_{p \in R} (j - j_b)^2$;
* $M^{'}_{1,1} = \sum_{p \in R} (i - i_b)(j - j_b)$;
* $M^{'}_{2,0} = \sum_{p \in R} (i - i_b)^2$.

then, we will have:
$$M(\bar{l}) = \alpha^2 M^{'}_{0,2} - 2 \alpha \beta M^{'}_{1,1} + \beta^2 M^{'}_{2,0}$$
*Note*: $\alpha$ and $\beta$ are variable, the moments are constant.

Since:
$$\begin{cases} \alpha = - \sin \theta \\ \beta = \cos \theta \end{cases}$$
then, we'll finally have:
$$M(\theta) = \sin^2 \theta M^{'}_{0,2} - 2 \sin \theta \cos \theta  M^{'}_{1,1} + \cos^2 \theta M^{'}_{2,0}$$

Therefore, finding the major axis boils down to determining the minimum of $M(\theta)$. 
A necessary condition to determine the minimum consists in the first derivative being zero:
$$\frac{dM(\theta)}{d \theta} = 2 \sin \theta \cos \theta M^{'}_{0,2} + 2 (\cos^2 \theta - \sin^2 \theta) M^{'}_{1,1} - 2 \sin \theta \cos \theta M^{'}_{2,0}$$
$$\frac{dM(\theta)}{d \theta} = \sin 2 \theta M^{'}_{0,2} + 2 \cos 2 \theta M^{'}_{1,1} - \sin 2 \theta M^{'}_{2,0}$$
$$\frac{dM(\theta)}{d \theta} = \sin 2 \theta (M^{'}_{0,2} - M^{'}_{2,0})+ 2 \cos 2 \theta M^{'}_{1,1}$$

We impose the null derivative $\frac{dM(\theta)}{d \theta} =  0$ and we'll have:
$$\tan 2 \theta = - \frac{2 M^{'}_{1,1}}{M^{'}_{0,2} - M^{'}_{2,0}}$$

The positive solution, that provides the major axis (minimum) will be:
$$2 \theta = \arctan \Biggl( - \frac{2 M^{'}_{1,1}}{M^{'}_{0,2} - M^{'}_{2,0}} \Biggr) = - \arctan \Biggl(\frac{2 M^{'}_{1,1}}{M^{'}_{0,2} - M^{'}_{2,0}} \Biggr) $$

$$\theta = - \frac{1}{2} \arctan \Biggl(\frac{2 M^{'}_{1,1}}{M^{'}_{0,2} - M^{'}_{2,0}} \Biggr)$$

The negative solution, that provides the minor axis (maximum) will be:
$$2 \theta = - \arctan \Biggl(\frac{2 M^{'}_{1,1}}{M^{'}_{0,2} - M^{'}_{2,0}} \Biggr) + \pi$$

$$\theta = - \frac{1}{2} \arctan \Biggl(\frac{2 M^{'}_{1,1}}{M^{'}_{0,2} - M^{'}_{2,0}} \Biggr) + \frac{\pi}{2}$$

Major and minor axes can be expressed into the image reference frame:

<center> <img src=https://i.ibb.co/T15PFq9/photo-2021-01-24-10-48-44.jpg width="1000px" /> </center>





## Oriented enclosing rectangle
Given two axes we might wish to draw a bounding box aligned to the object, referred as **oriented MER** (minimum enclosing rectangle).
To make it so, the four points laying at the maximum distance on opposite sides of the two axes must be determined:
<center> <img src=https://i.ibb.co/16H8PYK/photo-2021-01-24-11-41-05.jpg width="1400px" /> </center>

Some features related to the MER are:
* **Lenght**: extent of the object along the major axis $$L = d_{v_1 v_2} = d_{v_3 v_4} = \sqrt{(i_{v_1} - i_{v_2})^2 + (j_{v_1} - j_{v_2})^2} = \sqrt{(i_{v_3} - i_{v_4})^2 + (j_{v_3} - j_{v_4})^2}$$
* **Width**: extent of the object along the minor axis $$W = d_{v_1 v_3} = d_{v_2 v_4} = \sqrt{(i_{v_1} - i_{v_3})^2 + (j_{v_1} - j_{v_3})^2} = \sqrt{(i_{v_2} - i_{v_4})^2 + (j_{v_2} - j_{v_4})^2}$$
* **Elongatedess**
$$E = \frac{L}{W}$$
* **Rectangularity**
$$R = \frac{\text{object area}}{\text{oriented MER area}} = \frac{A}{LW}$$
* **Ellipticity**
$$E = \frac{A}{A_{LW}} \quad A_{LW} = \frac{\pi}{4} LW$$

*Note*: computation of $L$, $W$ and $R$ based on the MER doesn't provide invariance to rotation. 
To get it, the quantities must be computed based an oriented MER.